In [292]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [293]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
import tensorboard
import shutil

from datetime import datetime as dt

In [294]:
train_csv = pd.read_csv("/kaggle/input/osic-pulmonary-fibrosis-progression/train.csv")
test_csv = pd.read_csv("/kaggle/input/osic-pulmonary-fibrosis-progression/test.csv")

train_csv['Patient_Week'] = train_csv['Patient'].astype(str) + '_' + train_csv['Weeks'].astype(str)

train_data = train_csv.groupby("Patient")
# print(len(train_data))

unique_patients = train_csv['Patient'].unique()
# print(unique_patients)

for patient_id in unique_patients:
    print(train_csv[train_csv['Patient']==patient_id].iloc[0])
    break

Patient             ID00007637202177411956430
Weeks                                      -4
FVC                                      2315
Percent                               58.2536
Age                                        79
Sex                                      Male
SmokingStatus                       Ex-smoker
Patient_Week     ID00007637202177411956430_-4
Name: 0, dtype: object


In [295]:
# Need to add base fvc column, base percentage
train_csv['Weeks'] = train_csv['Weeks'].astype(int)
train_csv['base_week'] = train_csv.groupby('Patient')['Weeks'].transform('min')
train_csv['weeks_passed'] = train_csv['Weeks'] - train_csv['base_week']
train_csv['base_fvc'] = 0
train_csv['base_percent'] = 0

for index in range(len(train_csv)):
    base_data = train_csv[train_csv['Patient']==train_csv.iloc[index]['Patient']].iloc[0]
    train_csv.loc[index, 'base_fvc'] = base_data['FVC']
    train_csv.loc[index, 'base_percent'] = base_data['Percent']
    
train_csv

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Patient_Week,base_week,weeks_passed,base_fvc,base_percent
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,ID00007637202177411956430_-4,-4,0,2315,58.253649
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,ID00007637202177411956430_5,-4,9,2315,58.253649
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,ID00007637202177411956430_7,-4,11,2315,58.253649
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,ID00007637202177411956430_9,-4,13,2315,58.253649
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,ID00007637202177411956430_11,-4,15,2315,58.253649
...,...,...,...,...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked,ID00426637202313170790466_13,0,13,2925,71.824968
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked,ID00426637202313170790466_19,0,19,2925,71.824968
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked,ID00426637202313170790466_31,0,31,2925,71.824968
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked,ID00426637202313170790466_43,0,43,2925,71.824968


In [296]:
# X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = pd.get_dummies(data=X_data, drop_first=True)
selected_columns = ['Age', 'weeks_passed', 'base_fvc', 'base_percent']
X_data[selected_columns] = train_csv[selected_columns]
y_data = train_csv['FVC']

X_train = X_data.sample(frac=0.8,random_state=0)
X_test = X_data.drop(X_train.index)

y_train = y_data.sample(frac=0.8,random_state=0).astype(float)
y_test = y_data.drop(y_train.index).astype(float)

train_stats = X_train.describe().transpose()

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']


normalized_X_train = norm(X_train)
normalized_X_test = norm(X_test)

In [297]:
# sns.pairplot(X_train[['Age', 'weeks_passed', 'base_fvc', 'base_percent']], diag_kind="kde")

In [298]:
def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = - np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)

In [299]:
# def log_custom_acc(y_test, y_pred):
#     sd_clipped = tf.maximum(tf.math.reduce_std(y_pred), 70)
#     delta = tf.minimum(tf.math.abs(tf.math.subtract(y_test, y_pred)), 1000)
    
#     metric = - tf.math.sqrt(2) * delta / sd_clipped - tf.math.log(np.sqrt(2) * sd_clipped)
#     return metric

def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [300]:
# Loss - laplace likelihood

def log_custom_loss(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    
    fvc_pred = y_pred
    sigma = tf.math.reduce_std(y_pred)
    
    sigma_clip = tf.maximum(sigma, 70)
    delta = tf.abs(y_true - fvc_pred)
    delta = tf.minimum(delta, 1000)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32))
    metric = (delta / sigma_clip) * sq2 + tf.math.log(sigma_clip * sq2)
    return K.mean(metric)

In [301]:
# For quantile
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [302]:
def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dropout(0.25),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)
#   quantile=0.5
  
#   loss=tf.keras.losses.MeanSquaredLogarithmicError()
#   loss=lambda y,f: tilted_loss(quantile,y,f)

  model.compile(loss=log_custom_loss,
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()

In [303]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 128)               1024      
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_92 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_93 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_94 (Dense)             (None, 1)                 65        
Total params: 13,505
Trainable params: 13,505
Non-trainable params: 0
_________________________________________________

In [304]:
shutil.rmtree('logs')

In [305]:
EPOCHS = 1000

time_now = dt.now().strftime("%Y%m%d_%H%M%S")
my_callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='logs/'+str(time_now)),
]

history = model.fit(
  normalized_X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=my_callbacks)

Epoch 1/1000
31/31 [==============================] - 0s 10ms/step - loss: 24.7830 - mae: 2708.0024 - mse: 8016745.0000 - val_loss: 24.7981 - val_mae: 2650.0027 - val_mse: 7759155.0000
Epoch 2/1000
31/31 [==============================] - 0s 2ms/step - loss: 24.7827 - mae: 2707.4529 - mse: 8013999.5000 - val_loss: 24.7981 - val_mae: 2649.1904 - val_mse: 7755283.5000
Epoch 3/1000
31/31 [==============================] - 0s 2ms/step - loss: 24.7824 - mae: 2706.4390 - mse: 8009085.0000 - val_loss: 24.7981 - val_mae: 2647.8596 - val_mse: 7749126.5000
Epoch 4/1000
31/31 [==============================] - 0s 2ms/step - loss: 24.7816 - mae: 2704.2249 - mse: 7998750.0000 - val_loss: 24.7981 - val_mae: 2645.6558 - val_mse: 7739169.0000
Epoch 5/1000
31/31 [==============================] - 0s 2ms/step - loss: 24.7801 - mae: 2700.8696 - mse: 7982975.5000 - val_loss: 24.7981 - val_mae: 2641.5481 - val_mse: 7721151.0000
Epoch 6/1000
31/31 [==============================] - 0s 2ms/step - loss: 24.77

Epoch 46/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.7074 - mae: 409.2881 - mse: 282154.7812 - val_loss: 7.4413 - val_mae: 267.4924 - val_mse: 127850.9922
Epoch 47/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.6608 - mae: 377.9758 - mse: 229987.1406 - val_loss: 7.4030 - val_mae: 251.4562 - val_mse: 106475.9688
Epoch 48/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.6424 - mae: 367.1717 - mse: 216070.9531 - val_loss: 7.3887 - val_mae: 241.6780 - val_mse: 90262.2734
Epoch 49/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.6016 - mae: 344.3615 - mse: 192309.1094 - val_loss: 7.3667 - val_mae: 235.2103 - val_mse: 86545.1016
Epoch 50/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.6082 - mae: 340.9346 - mse: 185314.8750 - val_loss: 7.3354 - val_mae: 225.1016 - val_mse: 81632.8672
Epoch 51/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.5812 - mae: 340.4002 - mse: 190

31/31 [==============================] - 0s 2ms/step - loss: 7.4580 - mae: 294.9939 - mse: 155450.3750 - val_loss: 7.2344 - val_mae: 166.4652 - val_mse: 53485.6328
Epoch 93/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4894 - mae: 309.6938 - mse: 179181.5469 - val_loss: 7.2264 - val_mae: 172.0311 - val_mse: 58994.6250
Epoch 94/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4792 - mae: 304.3188 - mse: 176397.5312 - val_loss: 7.2210 - val_mae: 191.4646 - val_mse: 81094.1484
Epoch 95/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4761 - mae: 302.6004 - mse: 166630.3594 - val_loss: 7.2129 - val_mae: 182.7707 - val_mse: 73646.9219
Epoch 96/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4896 - mae: 303.7440 - mse: 168061.2969 - val_loss: 7.2238 - val_mae: 176.6611 - val_mse: 65524.3242
Epoch 97/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4798 - mae: 297.2734 - mse: 164057.2500 - val_l

Epoch 138/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4640 - mae: 299.0419 - mse: 183889.2031 - val_loss: 7.2118 - val_mae: 214.6205 - val_mse: 149838.9844
Epoch 139/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4533 - mae: 306.2539 - mse: 217585.0938 - val_loss: 7.2092 - val_mae: 198.2829 - val_mse: 124761.1797
Epoch 140/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4788 - mae: 312.2127 - mse: 226445.4688 - val_loss: 7.2091 - val_mae: 187.0871 - val_mse: 91492.7656
Epoch 141/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4677 - mae: 290.9459 - mse: 170561.1094 - val_loss: 7.2214 - val_mae: 197.8440 - val_mse: 94144.0312
Epoch 142/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4859 - mae: 313.4396 - mse: 195154.4844 - val_loss: 7.2116 - val_mae: 182.5952 - val_mse: 82607.5469
Epoch 143/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4670 - mae: 298.1476 - ms

31/31 [==============================] - 0s 2ms/step - loss: 7.4690 - mae: 305.1780 - mse: 212031.7656 - val_loss: 7.2046 - val_mae: 198.4734 - val_mse: 143510.7812
Epoch 184/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4508 - mae: 297.8053 - mse: 219812.8281 - val_loss: 7.2136 - val_mae: 204.7075 - val_mse: 131031.8672
Epoch 185/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4697 - mae: 302.0244 - mse: 188470.1719 - val_loss: 7.2036 - val_mae: 198.1221 - val_mse: 126329.9688
Epoch 186/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4390 - mae: 294.3502 - mse: 206838.9688 - val_loss: 7.2056 - val_mae: 181.8045 - val_mse: 96733.4375
Epoch 187/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4568 - mae: 297.7775 - mse: 186922.0312 - val_loss: 7.2100 - val_mae: 201.4909 - val_mse: 131247.7031
Epoch 188/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.4687 - mae: 309.4091 - mse: 225651.890

...

31/31 [==============================] - 0s 2ms/step - loss: 7.3887 - mae: 274.4389 - mse: 181684.4531 - val_loss: 7.2120 - val_mae: 190.8139 - val_mse: 102519.9688
Epoch 818/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3823 - mae: 267.3559 - mse: 168161.8281 - val_loss: 7.2246 - val_mae: 200.1017 - val_mse: 121207.0391
Epoch 819/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3722 - mae: 272.4156 - mse: 187389.4375 - val_loss: 7.2184 - val_mae: 208.3691 - val_mse: 147328.4062
Epoch 820/1000
31/31 [==============================] - 0s 3ms/step - loss: 7.3737 - mae: 271.3198 - mse: 183388.0000 - val_loss: 7.2282 - val_mae: 227.9066 - val_mse: 183631.3281
Epoch 821/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3755 - mae: 268.4867 - mse: 184552.7188 - val_loss: 7.2184 - val_mae: 210.9530 - val_mse: 145317.0156
Epoch 822/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3745 - mae: 262.5652 - mse: 158347.10

31/31 [==============================] - 0s 2ms/step - loss: 7.3587 - mae: 261.7916 - mse: 176204.8906 - val_loss: 7.2605 - val_mae: 247.8917 - val_mse: 199701.0312
Epoch 863/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3550 - mae: 267.1675 - mse: 188461.8438 - val_loss: 7.2075 - val_mae: 194.8378 - val_mse: 124006.4922
Epoch 864/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3766 - mae: 267.5555 - mse: 185772.6094 - val_loss: 7.2214 - val_mae: 219.1736 - val_mse: 155617.5625
Epoch 865/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3486 - mae: 261.1692 - mse: 192905.4062 - val_loss: 7.2048 - val_mae: 195.8228 - val_mse: 124084.0547
Epoch 866/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3819 - mae: 274.4554 - mse: 184170.2812 - val_loss: 7.2120 - val_mae: 196.5868 - val_mse: 113002.2578
Epoch 867/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3370 - mae: 250.0664 - mse: 155573.96

31/31 [==============================] - 0s 2ms/step - loss: 7.3497 - mae: 259.5055 - mse: 190403.6250 - val_loss: 7.2105 - val_mae: 210.8521 - val_mse: 164996.0312
Epoch 908/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3569 - mae: 262.9485 - mse: 196179.6094 - val_loss: 7.2329 - val_mae: 192.0748 - val_mse: 102432.4141
Epoch 909/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3649 - mae: 261.9998 - mse: 152010.7031 - val_loss: 7.2080 - val_mae: 194.5842 - val_mse: 115671.9453
Epoch 910/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3720 - mae: 263.7626 - mse: 187690.0000 - val_loss: 7.2155 - val_mae: 195.4679 - val_mse: 104456.4531
Epoch 911/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3772 - mae: 272.2942 - mse: 168129.6250 - val_loss: 7.2130 - val_mae: 202.6660 - val_mse: 133068.9844
Epoch 912/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3677 - mae: 266.6292 - mse: 177311.34

31/31 [==============================] - 0s 2ms/step - loss: 7.3491 - mae: 263.1908 - mse: 193588.3750 - val_loss: 7.2387 - val_mae: 225.7947 - val_mse: 141007.9844
Epoch 953/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3548 - mae: 266.0664 - mse: 182740.1406 - val_loss: 7.2211 - val_mae: 200.6205 - val_mse: 109583.2734
Epoch 954/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3412 - mae: 253.8331 - mse: 161063.6719 - val_loss: 7.2201 - val_mae: 196.8188 - val_mse: 107444.9922
Epoch 955/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3489 - mae: 259.5153 - mse: 163791.6875 - val_loss: 7.2157 - val_mae: 208.8445 - val_mse: 141694.5469
Epoch 956/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3427 - mae: 259.4149 - mse: 178135.9219 - val_loss: 7.2144 - val_mae: 200.7038 - val_mse: 126710.8672
Epoch 957/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3460 - mae: 266.8225 - mse: 203141.28

31/31 [==============================] - 0s 2ms/step - loss: 7.3516 - mae: 264.3395 - mse: 180854.6094 - val_loss: 7.2295 - val_mae: 226.4603 - val_mse: 165237.7812
Epoch 998/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3479 - mae: 257.5506 - mse: 173708.8281 - val_loss: 7.2064 - val_mae: 199.6130 - val_mse: 126906.8516
Epoch 999/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3574 - mae: 266.8758 - mse: 195066.2344 - val_loss: 7.2202 - val_mae: 211.1454 - val_mse: 136631.8281
Epoch 1000/1000
31/31 [==============================] - 0s 2ms/step - loss: 7.3338 - mae: 251.3967 - mse: 165891.5000 - val_loss: 7.2134 - val_mae: 204.7911 - val_mse: 126785.5547


In [306]:
def inference(model, X_test,y_test):
    y_pred = model.predict(X_test).reshape(-1)
    return y_pred, laplace_log_likelihood(y_test, y_pred, np.std(y_pred))

y_pred, metric = inference(model, normalized_X_test, y_test.tolist())
print(metric)

-7.255216631815051


In [307]:
np.std(y_pred)

664.23676

In [308]:
loss, mae, mse = model.evaluate(normalized_X_test, y_test, verbose=2)

10/10 - 0s - loss: 7.2311 - mae: 193.1203 - mse: 69072.0547
